<a href="https://colab.research.google.com/github/Knightler/deep-learning-practice/blob/main/GNN_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch

# Number of nodes
num_nodes = 4

# Node features: (4 nodes, 3 features each)
X = torch.tensor([
    [1.0, 0.5, 0.2],   # Node 0
    [0.9, 0.7, 0.3],   # Node 1
    [0.4, 0.3, 0.9],   # Node 2
    [0.6, 0.1, 0.8]    # Node 3
], dtype=torch.float32)

# Adjacency matrix (undirected graph)
A = torch.tensor([
    [1, 1, 1, 0],  # Node 0 connected to 1, 2 (and itself)
    [1, 1, 0, 1],  # Node 1 connected to 0, 3
    [1, 0, 1, 1],  # Node 2 connected to 0, 3
    [0, 1, 1, 1]   # Node 3 connected to 1, 2
], dtype=torch.float32)


In [17]:
D_inv_sqrt = torch.diag(1.0 / torch.sqrt(A.sum(1)))
A_hat = D_inv_sqrt @ A @ D_inv_sqrt  # normalized adjacency

In [18]:
# Trainable weight matrix: turns 3 features → 2 features
W = torch.nn.Parameter(torch.randn(3, 2))

# GCN layer function
def gcn_layer(X, A_hat, W):
    return A_hat @ X @ W

In [19]:
# Run the GCN layer
output = gcn_layer(X, A_hat, W)

print("Node embeddings after 1 GCN layer:")
print(output)

Node embeddings after 1 GCN layer:
tensor([[-0.1486,  0.1270],
        [-0.1044,  0.1264],
        [-0.4163,  0.0370],
        [-0.4494,  0.0418]], grad_fn=<MmBackward0>)


In [20]:
import torch.nn.functional as F

# Second layer weight: 2 input → 2 output (new embedding)
W2 = torch.nn.Parameter(torch.randn(2, 2))

# Two-layer GCN
def gcn_two_layers(X, A_hat, W1, W2):
    H1 = A_hat @ X @ W1     # First GCN layer
    H1 = F.relu(H1)         # Non-linearity
    H2 = A_hat @ H1 @ W2    # Second GCN layer
    return H2

In [21]:
output = gcn_two_layers(X, A_hat, W, W2)

print("Final node embeddings after 2 GCN layers:")
print(output)

Final node embeddings after 2 GCN layers:
tensor([[0.0447, 0.0259],
        [0.0454, 0.0263],
        [0.0317, 0.0183],
        [0.0316, 0.0183]], grad_fn=<MmBackward0>)


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(in_features, out_features))

    def forward(self, X, A_hat):
        out = A_hat @ X @ self.W
        return out

In [23]:
class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_features, hidden_features)
        self.gcn2 = GCNLayer(hidden_features, out_features)

    def forward(self, X, A_hat):
        H1 = F.relu(self.gcn1(X, A_hat))
        H2 = self.gcn2(H1, A_hat)
        return H2

In [24]:
model = GCN(in_features=3, hidden_features=4, out_features=2)
out = model(X, A_hat)

print("Node embeddings:")
print(out)

Node embeddings:
tensor([[ 1.2590, -0.4594],
        [ 1.2769, -0.4668],
        [ 1.2223, -0.4673],
        [ 1.2134, -0.4623]], grad_fn=<MmBackward0>)


In [25]:
# True classes for each node (0 or 1)
labels = torch.tensor([0, 0, 1, 1])

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [27]:
torch.manual_seed(42)  # Set before model init

In [29]:
for epoch in range(200):
    optimizer.zero_grad()
    out = model(X, A_hat)           # Forward pass
    loss = criterion(out, labels)   # Compute loss
    loss.backward()                 # Backprop
    optimizer.step()                # Update weights

    if epoch % 10 == 0:
        pred = out.argmax(dim=1)
        acc = (pred == labels).float().mean()
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Accuracy: {acc.item():.4f}")

Epoch 0, Loss: 0.6997, Accuracy: 0.5000
Epoch 10, Loss: 0.6972, Accuracy: 0.5000
Epoch 20, Loss: 0.6946, Accuracy: 0.5000
Epoch 30, Loss: 0.6922, Accuracy: 0.5000
Epoch 40, Loss: 0.6900, Accuracy: 0.5000
Epoch 50, Loss: 0.6875, Accuracy: 0.5000
Epoch 60, Loss: 0.6845, Accuracy: 0.5000
Epoch 70, Loss: 0.6808, Accuracy: 0.5000
Epoch 80, Loss: 0.6761, Accuracy: 0.5000
Epoch 90, Loss: 0.6699, Accuracy: 0.5000
Epoch 100, Loss: 0.6620, Accuracy: 1.0000
Epoch 110, Loss: 0.6519, Accuracy: 1.0000
Epoch 120, Loss: 0.6396, Accuracy: 1.0000
Epoch 130, Loss: 0.6262, Accuracy: 1.0000
Epoch 140, Loss: 0.6112, Accuracy: 1.0000
Epoch 150, Loss: 0.5946, Accuracy: 1.0000
Epoch 160, Loss: 0.5760, Accuracy: 1.0000
Epoch 170, Loss: 0.5560, Accuracy: 1.0000
Epoch 180, Loss: 0.5347, Accuracy: 1.0000
Epoch 190, Loss: 0.5120, Accuracy: 1.0000


In [30]:
# New node feature (same 3 features as original input)
new_feature = torch.tensor([[0.5, 0.6, 0.1]])
X_new = torch.cat([X, new_feature], dim=0)

In [31]:
A_new = torch.zeros((5, 5))
A_new[:4, :4] = A  # Copy old adjacency
A_new[4, 1] = 1    # New connections
A_new[1, 4] = 1
A_new[4, 2] = 1
A_new[2, 4] = 1
A_new[4, 4] = 1    # self-connection

In [32]:
D_inv_sqrt_new = torch.diag(1.0 / torch.sqrt(A_new.sum(1)))
A_hat_new = D_inv_sqrt_new @ A_new @ D_inv_sqrt_new

In [33]:
# Use the old model
output_new = model(X_new, A_hat_new)
print("Embeddings after adding new node:")
print(output_new)

Embeddings after adding new node:
tensor([[ 0.9995, -0.2479],
        [ 0.9458, -0.1198],
        [ 0.6439,  0.1171],
        [ 0.5047,  0.1435],
        [ 0.9675, -0.2406]], grad_fn=<MmBackward0>)


In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ==== 1. Define GCN Layer and Model ====

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(in_features, out_features))

    def forward(self, X, A_hat):
        return A_hat @ X @ self.W

class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_features, hidden_features)
        self.gcn2 = GCNLayer(hidden_features, out_features)

    def forward(self, X, A_hat):
        H1 = F.relu(self.gcn1(X, A_hat))
        H2 = self.gcn2(H1, A_hat)
        return H2

# ==== 2. Initial Graph: 4 Chat Nodes ====

X = torch.tensor([
    [0.9, 0.2, 0.7],  # Chat 0 (tech)
    [0.8, 0.1, 0.9],  # Chat 1 (tech)
    [0.3, 0.7, 0.2],  # Chat 2 (personal)
    [0.2, 0.8, 0.1],  # Chat 3 (personal)
], dtype=torch.float32)

A = torch.tensor([
    [1, 1, 0, 0],
    [1, 1, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 1, 1],
], dtype=torch.float32)

# Normalize adjacency
D_inv_sqrt = torch.diag(1.0 / torch.sqrt(A.sum(1)))
A_hat = D_inv_sqrt @ A @ D_inv_sqrt

# Labels: 0 for tech, 1 for personal
labels = torch.tensor([0, 0, 1, 1])

# ==== 3. Train GCN ====

model = GCN(in_features=3, hidden_features=4, out_features=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

for epoch in range(200):
    optimizer.zero_grad()
    out = model(X, A_hat)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()

# ==== 4. Add New Nodes ====

# Two new chats (1 tech, 1 personal)
new_chats = torch.tensor([
    [0.85, 0.15, 0.8],    # new tech-like
    [0.25, 0.75, 0.05]    # new personal-like
], dtype=torch.float32)

X_new = torch.cat([X, new_chats], dim=0)

# New adjacency matrix (6x6)
A_new = torch.zeros((6, 6))
A_new[:4, :4] = A
A_new[4, 1] = 1  # New tech chat ↔ Chat 1
A_new[1, 4] = 1
A_new[5, 3] = 1  # New personal chat ↔ Chat 3
A_new[3, 5] = 1
A_new[4, 4] = 1  # self
A_new[5, 5] = 1

# Normalize new adjacency
D_inv_sqrt_new = torch.diag(1.0 / torch.sqrt(A_new.sum(1)))
A_hat_new = D_inv_sqrt_new @ A_new @ D_inv_sqrt_new

# ==== 5. Forward Pass with New Nodes ====
output_new = model(X_new, A_hat_new)

output_new

tensor([[ 3.5919, -2.0364],
        [ 4.4036, -2.5882],
        [ 0.8962,  4.4599],
        [ 1.0832,  5.3998],
        [ 3.6282, -2.1731],
        [ 0.8803,  4.4043]], grad_fn=<MmBackward0>)

In [37]:
chats = [
    "How do I fine-tune a language model?",
    "Tell me about loss functions in deep learning.",
    "What's a healthy meal for weight loss?",
    "How to stay motivated while studying?",
]

In [38]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')

X = embedder.encode(chats, convert_to_tensor=True)  # shape: (n_chats, dim)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

similarity = cosine_similarity(X)
threshold = 0.6
A = (similarity > threshold).astype(float)
np.fill_diagonal(A, 1)  # self-connection
A = torch.tensor(A, dtype=torch.float32)

In [41]:
model = GCN(in_features=384, hidden_features=64, out_features=32)

In [42]:
D_inv_sqrt = torch.diag(1.0 / torch.sqrt(A.sum(1)))
A_hat = D_inv_sqrt @ A @ D_inv_sqrt

output = model(X, A_hat)